In [2]:
library("lme4")
library("ggplot2")
library("dplyr")
library(MASS)
library("BayesFactor")
library("car")
library("scales")
library("lmerTest")
library("MuMIn")
library("plyr")
library("rstatix")
library("ggpubr")
library("knitr")
library("corrplot")
library("RColorBrewer")

In [3]:
#Load the data => Here, we are filtering the cases in which subjects had a MuIFI of 0
FinalPath<-'../ToInput/MS17temp_R_HumMod_compare_t02_All_wCV.txt'
data <- read.delim(FinalPath, sep="\t", header=T, stringsAsFactors=F)
data <- data %>% filter(!Amplitude==0)
dataTest <- data
data$lCV <- log(data$CV)
NArow <- which(is.na(data$lCV))
data <- data %>% filter(!is.na(lCV))

In [4]:
##Source for "SummarySE": http://www.cookbook-r.com/Manipulating_data/Summarizing_data/
##
##
## Gives count, mean, standard deviation, standard error of the mean, and confidence interval (default 95%).
##   data: a data frame.
##   measurevar: the name of a column that contains the variable to be summariezed
##   groupvars: a vector containing names of columns that contain grouping variables
##   na.rm: a boolean that indicates whether to ignore NA's
##   conf.interval: the percent range of the confidence interval (default is 95%)
summarySE <- function(data=NULL, measurevar, groupvars=NULL, na.rm=FALSE,
                      conf.interval=.95, .drop=TRUE) {
    library(plyr)

    # New version of length which can handle NA's: if na.rm==T, don't count them
    length2 <- function (x, na.rm=FALSE) {
        if (na.rm) sum(!is.na(x))
        else       length(x)
    }

    # This does the summary. For each group's data frame, return a vector with
    # N, mean, and sd
    datac <- ddply(data, groupvars, .drop=.drop,
      .fun = function(xx, col) {
        c(N    = length2(xx[[col]], na.rm=na.rm),
          mean = mean   (xx[[col]], na.rm=na.rm),
          sd   = sd     (xx[[col]], na.rm=na.rm)
        )
      },
      measurevar
    )

    # Rename the "mean" column    
    datac <- rename(datac, c("mean" = measurevar))

    datac$se <- datac$sd / sqrt(datac$N)  # Calculate standard error of the mean

    # Confidence interval multiplier for standard error
    # Calculate t-statistic for confidence interval: 
    # e.g., if conf.interval is .95, use .975 (above/below), and use df=N-1
    ciMult <- qt(conf.interval/2 + .5, datac$N-1)
    datac$ci <- datac$se * ciMult

    return(datac)
}

In [5]:
# Multiple plot function: source: http://www.cookbook-r.com/Graphs/Multiple_graphs_on_one_page_(ggplot2)/
#
#
# ggplot objects can be passed in ..., or to plotlist (as a list of ggplot objects)
# - cols:   Number of columns in layout
# - layout: A matrix specifying the layout. If present, 'cols' is ignored.
#
# If the layout is something like matrix(c(1,2,3,3), nrow=2, byrow=TRUE),
# then plot 1 will go in the upper left, 2 will go in the upper right, and
# 3 will go all the way across the bottom.
#
multiplot <- function(..., plotlist=NULL, file, cols=1, layout=NULL) {
  library(grid)

  # Make a list from the ... arguments and plotlist
  plots <- c(list(...), plotlist)

  numPlots = length(plots)

  # If layout is NULL, then use 'cols' to determine layout
  if (is.null(layout)) {
    # Make the panel
    # ncol: Number of columns of plots
    # nrow: Number of rows needed, calculated from # of cols
    layout <- matrix(seq(1, cols * ceiling(numPlots/cols)),
                    ncol = cols, nrow = ceiling(numPlots/cols))
  }

 if (numPlots==1) {
    print(plots[[1]])

  } else {
    # Set up the page
    grid.newpage()
    pushViewport(viewport(layout = grid.layout(nrow(layout), ncol(layout))))

    # Make each plot, in the correct location
    for (i in 1:numPlots) {
      # Get the i,j matrix positions of the regions that contain this subplot
      matchidx <- as.data.frame(which(layout == i, arr.ind = TRUE))

      print(plots[[i]], vp = viewport(layout.pos.row = matchidx$row,
                                      layout.pos.col = matchidx$col))
    }
  }
}

In [6]:
table(data$GameSpeed, data$Temperature)

     
      Human   no  yes
  MMM   273 1499 1499
  MSM   286 1490 1491
  SMS   282 1495 1498
  SSS   273 1499 1496

# Compute r and RMSE

In [9]:
#RMSE function definition
RMSE = function(mod, obs){
  sqrt(mean((mod - obs)^2))
}

#Filtered data set
filt_data <- data

In [10]:
cont_table <- ddply(filt_data, c("Subject","Temperature","GameNb","GameSpeed"), summarize,  Amplitude=mean(Amplitude))
table(cont_table$Temperature,cont_table$GameSpeed)

       
         MMM  MSM  SMS  SSS
  Human  273  286  282  273
  no    1499 1490 1495 1499
  yes   1499 1491 1498 1496

In [11]:
#1) Performance results - no temperature & Humans
muTab<-aggregate(filt_data$Points, by=list(filt_data$Temperature,filt_data$GameSpeed,filt_data$GameNb), FUN=mean)
print("Table for Points")
colnames(muTab)<-c("Temperature","GameSpeed","GameNb","Value")
H_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    H_Mat[i,] <- muTab[which(muTab$Temperature=="Human" & muTab$GameNb==i),]$Value
}
M1_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    M1_Mat[i,] <- muTab[which(muTab$Temperature=="yes" & muTab$GameNb==i),]$Value
}
M2_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    M2_Mat[i,] <- muTab[which(muTab$Temperature=="no" & muTab$GameNb==i),]$Value
}
colnames(H_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(H_Mat) <- 1:15
colnames(M1_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(M1_Mat) <- 1:15
colnames(M2_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(M2_Mat) <- 1:15

#correlations & RMSE per condition
#print("Performance correlations & RMSE")
condition <- c("MMM","MSM","SMS","SSS")
rmse <- rep(0,4)
for (cond in 1:4) {
    rmse[cond] <- RMSE(M1_Mat[,cond],H_Mat[,cond])
}
rmse_same <- rmse
rmse_same
rmse <- rep(0,4)
for (cond in 1:4) {
    rmse[cond] <- RMSE(M2_Mat[,cond],H_Mat[,cond])
}
rmse_diff <- rmse
rmse_diff

print("Overall correlation & RMSE")
M1_avg <- mean(rmse_same)
M2_avg <- mean(rmse_diff)
M1_avg
M2_avg

#Difference avg rmse delta - positive means keeping temperature is better
Perf_delta <- M2_avg - M1_avg
Perf_delta

[1] "Table for Points"


[1] 506.3334 227.9582 217.5296 117.8321

[1] 507.1129 226.0335 244.6830 145.2561

[1] "Overall correlation & RMSE"


[1] 267.4133

[1] 280.7714

[1] 13.35807

In [12]:
#2) Entropy results - no temperature & Humans
muTab<-aggregate(filt_data$Entropy, by=list(filt_data$Temperature,filt_data$GameSpeed,filt_data$GameNb), FUN=mean)
print("Table for Entropy")
colnames(muTab)<-c("Temperature","GameSpeed","GameNb","Value")
H_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    H_Mat[i,] <- muTab[which(muTab$Temperature=="Human" & muTab$GameNb==i),]$Value
}
M1_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    M1_Mat[i,] <- muTab[which(muTab$Temperature=="yes" & muTab$GameNb==i),]$Value
}
M2_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    M2_Mat[i,] <- muTab[which(muTab$Temperature=="no" & muTab$GameNb==i),]$Value
}
colnames(H_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(H_Mat) <- 1:15
colnames(M1_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(M1_Mat) <- 1:15
colnames(M2_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(M2_Mat) <- 1:15


#correlations & RMSE per condition
#print("Performance correlations & RMSE")
condition <- c("MMM","MSM","SMS","SSS")
rmse <- rep(0,4)
for (cond in 1:4) {
    rmse[cond] <- RMSE(M1_Mat[,cond],H_Mat[,cond])
}
rmse_same <- rmse
rmse_same
rmse <- rep(0,4)
for (cond in 1:4) {
    rmse[cond] <- RMSE(M2_Mat[,cond],H_Mat[,cond])
}
rmse_diff <- rmse
rmse_diff

print("Overall correlation & RMSE")
M1_avg <- mean(rmse_same)
M2_avg <- mean(rmse_diff)
M1_avg
M2_avg

#Difference avg rmse delta - positive means keeping temperature is better
Ent_delta <- M2_avg - M1_avg
Ent_delta

[1] "Table for Entropy"


[1] 0.13297035 0.06945208 0.12864930 0.20811395

[1] 0.13160052 0.06704038 0.11053673 0.21270353

[1] "Overall correlation & RMSE"


[1] 0.1347964

[1] 0.1304703

[1] -0.004326131

In [13]:
#3) Log CV ISI results
muTab<-aggregate(filt_data$lCV, by=list(filt_data$Temperature,filt_data$GameSpeed,filt_data$GameNb), FUN=mean)
print("Table for log CV")
colnames(muTab)<-c("Temperature","GameSpeed","GameNb","Value")
H_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    H_Mat[i,] <- muTab[which(muTab$Temperature=="Human" & muTab$GameNb==i),]$Value
}
M1_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    M1_Mat[i,] <- muTab[which(muTab$Temperature=="yes" & muTab$GameNb==i),]$Value
}
M2_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    M2_Mat[i,] <- muTab[which(muTab$Temperature=="no" & muTab$GameNb==i),]$Value
}
colnames(H_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(H_Mat) <- 1:15
colnames(M1_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(M1_Mat) <- 1:15
colnames(M2_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(M2_Mat) <- 1:15


#correlations & RMSE per condition
#print("Performance correlations & RMSE")
condition <- c("MMM","MSM","SMS","SSS")
rmse <- rep(0,4)
for (cond in 1:4) {
    rmse[cond] <- RMSE(M1_Mat[,cond],H_Mat[,cond])
}
rmse_same <- rmse
rmse_same
rmse <- rep(0,4)
for (cond in 1:4) {
    rmse[cond] <- RMSE(M2_Mat[,cond],H_Mat[,cond])
}
rmse_diff <- rmse
rmse_diff

print("Overall correlation & RMSE")
M1_avg <- mean(rmse_same)
M2_avg <- mean(rmse_diff)
M1_avg
M2_avg

#Difference avg rmse delta - positive means keeping temperature is better
lCV_delta <- M2_avg - M1_avg
lCV_delta

[1] "Table for log CV"


[1] 0.4971121 0.2557528 0.2182773 0.4487135

[1] 0.4979796 0.2430459 0.2099198 0.4558655

[1] "Overall correlation & RMSE"


[1] 0.3549639

[1] 0.3517027

[1] -0.003261224

In [14]:
#4) Periodicity results
muTab<-aggregate(filt_data$Periodicity, by=list(filt_data$Temperature,filt_data$GameSpeed,filt_data$GameNb), FUN=mean)
print("Table for Periodicity")
colnames(muTab)<-c("Temperature","GameSpeed","GameNb","Value")
H_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    H_Mat[i,] <- muTab[which(muTab$Temperature=="Human" & muTab$GameNb==i),]$Value
}
M1_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    M1_Mat[i,] <- muTab[which(muTab$Temperature=="yes" & muTab$GameNb==i),]$Value
}
M2_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    M2_Mat[i,] <- muTab[which(muTab$Temperature=="no" & muTab$GameNb==i),]$Value
}
colnames(H_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(H_Mat) <- 1:15
colnames(M1_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(M1_Mat) <- 1:15
colnames(M2_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(M2_Mat) <- 1:15


#correlations & RMSE per condition
#print("Performance correlations & RMSE")
condition <- c("MMM","MSM","SMS","SSS")
rmse <- rep(0,4)
for (cond in 1:4) {
    rmse[cond] <- RMSE(M1_Mat[,cond],H_Mat[,cond])
}
rmse_same <- rmse
rmse_same
rmse <- rep(0,4)
for (cond in 1:4) {
    rmse[cond] <- RMSE(M2_Mat[,cond],H_Mat[,cond])
}
rmse_diff <- rmse
rmse_diff

print("Overall correlation & RMSE")
M1_avg <- mean(rmse_same)
M2_avg <- mean(rmse_diff)
M1_avg
M2_avg

#Difference avg rmse delta - positive means keeping temperature is better
Per_delta <- M2_avg - M1_avg
Per_delta

[1] "Table for Periodicity"


[1] 55.19018 67.78008 58.03874 80.64563

[1] 57.34365 62.98331 58.49662 80.74668

[1] "Overall correlation & RMSE"


[1] 65.41366

[1] 64.89256

[1] -0.5210925

In [16]:
#5) Regularity results
muTab<-aggregate(filt_data$Amplitude, by=list(filt_data$Temperature,filt_data$GameSpeed,filt_data$GameNb), FUN=mean)
print("Table for Regularity")
colnames(muTab)<-c("Temperature","GameSpeed","GameNb","Value")
H_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    H_Mat[i,] <- muTab[which(muTab$Temperature=="Human" & muTab$GameNb==i),]$Value
}
M1_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    M1_Mat[i,] <- muTab[which(muTab$Temperature=="yes" & muTab$GameNb==i),]$Value
}
M2_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    M2_Mat[i,] <- muTab[which(muTab$Temperature=="no" & muTab$GameNb==i),]$Value
}
colnames(H_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(H_Mat) <- 1:15
colnames(M1_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(M1_Mat) <- 1:15
colnames(M2_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(M2_Mat) <- 1:15


#correlations & RMSE per condition
#print("Performance correlations & RMSE")
condition <- c("MMM","MSM","SMS","SSS")
rmse <- rep(0,4)
for (cond in 1:4) {
    rmse[cond] <- RMSE(M1_Mat[,cond],H_Mat[,cond])
}
rmse_same <- rmse
rmse_same
rmse <- rep(0,4)
for (cond in 1:4) {
    rmse[cond] <- RMSE(M2_Mat[,cond],H_Mat[,cond])
}
rmse_diff <- rmse
rmse_diff

print("Overall correlation & RMSE")
M1_avg <- mean(rmse_same)
M2_avg <- mean(rmse_diff)
M1_avg
M2_avg

#Difference avg rmse delta - positive means keeping temperature is better
Reg_delta <- M2_avg - M1_avg
Reg_delta

[1] "Table for Regularity"


[1] 0.04042178 0.03266586 0.02588127 0.03465337

[1] 0.04149782 0.02576453 0.04391037 0.03863908

[1] "Overall correlation & RMSE"


[1] 0.03340557

[1] 0.03745295

[1] 0.004047378

In [15]:
#6) Resets results
muTab<-aggregate(filt_data$Resets, by=list(filt_data$Temperature,filt_data$GameSpeed,filt_data$GameNb), FUN=mean)
print("Table for Resets")
colnames(muTab)<-c("Temperature","GameSpeed","GameNb","Value")
H_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    H_Mat[i,] <- muTab[which(muTab$Temperature=="Human" & muTab$GameNb==i),]$Value
}
M1_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    M1_Mat[i,] <- muTab[which(muTab$Temperature=="yes" & muTab$GameNb==i),]$Value
}
M2_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    M2_Mat[i,] <- muTab[which(muTab$Temperature=="no" & muTab$GameNb==i),]$Value
}
colnames(H_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(H_Mat) <- 1:15
colnames(M1_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(M1_Mat) <- 1:15
colnames(M2_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(M2_Mat) <- 1:15


#correlations & RMSE per condition
#print("Performance correlations & RMSE")
condition <- c("MMM","MSM","SMS","SSS")
rmse <- rep(0,4)
for (cond in 1:4) {
    rmse[cond] <- RMSE(M1_Mat[,cond],H_Mat[,cond])
}
rmse_same <- rmse
rmse_same
rmse <- rep(0,4)
for (cond in 1:4) {
    rmse[cond] <- RMSE(M2_Mat[,cond],H_Mat[,cond])
}
rmse_diff <- rmse
rmse_diff

print("Overall correlation & RMSE")
M1_avg <- mean(rmse_same)
M2_avg <- mean(rmse_diff)
M1_avg
M2_avg

#Difference avg rmse delta - positive means keeping temperature is better
Res_delta <- M2_avg - M1_avg
Res_delta

[1] "Table for Resets"


[1] 1.882929 6.109646 6.221140 3.139026

[1] 2.413616 7.210624 5.728427 3.585598

[1] "Overall correlation & RMSE"


[1] 4.338185

[1] 4.734566

[1] 0.3963811

In [16]:
#7) Deflations results
muTab<-aggregate(filt_data$Deflations, by=list(filt_data$Temperature,filt_data$GameSpeed,filt_data$GameNb), FUN=mean)
print("Table for Deflations")
colnames(muTab)<-c("Temperature","GameSpeed","GameNb","Value")
H_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    H_Mat[i,] <- muTab[which(muTab$Temperature=="Human" & muTab$GameNb==i),]$Value
}
M1_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    M1_Mat[i,] <- muTab[which(muTab$Temperature=="yes" & muTab$GameNb==i),]$Value
}
M2_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    M2_Mat[i,] <- muTab[which(muTab$Temperature=="no" & muTab$GameNb==i),]$Value
}
colnames(H_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(H_Mat) <- 1:15
colnames(M1_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(M1_Mat) <- 1:15
colnames(M2_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(M2_Mat) <- 1:15

#correlations & RMSE per condition
#print("Performance correlations & RMSE")
condition <- c("MMM","MSM","SMS","SSS")
rmse <- rep(0,4)
for (cond in 1:4) {
    rmse[cond] <- RMSE(M1_Mat[,cond],H_Mat[,cond])
}
rmse_same <- rmse
rmse_same
rmse <- rep(0,4)
for (cond in 1:4) {
    rmse[cond] <- RMSE(M2_Mat[,cond],H_Mat[,cond])
}
rmse_diff <- rmse
rmse_diff

print("Overall correlation & RMSE")
M1_avg <- mean(rmse_same)
M2_avg <- mean(rmse_diff)
M1_avg
M2_avg

#Difference avg rmse delta - positive means keeping temperature is better
Def_delta <- M2_avg - M1_avg
Def_delta

[1] "Table for Deflations"


[1] 3.922770 4.106495 3.235468 1.217421

[1] 3.845649 4.382451 4.440100 1.229434

[1] "Overall correlation & RMSE"


[1] 3.120538

[1] 3.474408

[1] 0.35387

In [17]:
#8) Misses results
muTab<-aggregate(filt_data$Misses, by=list(filt_data$Temperature,filt_data$GameSpeed,filt_data$GameNb), FUN=mean)
print("Table for Misses")
colnames(muTab)<-c("Temperature","GameSpeed","GameNb","Value")
H_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    H_Mat[i,] <- muTab[which(muTab$Temperature=="Human" & muTab$GameNb==i),]$Value
}
M1_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    M1_Mat[i,] <- muTab[which(muTab$Temperature=="yes" & muTab$GameNb==i),]$Value
}
M2_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    M2_Mat[i,] <- muTab[which(muTab$Temperature=="no" & muTab$GameNb==i),]$Value
}
colnames(H_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(H_Mat) <- 1:15
colnames(M1_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(M1_Mat) <- 1:15
colnames(M2_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(M2_Mat) <- 1:15

#correlations & RMSE per condition
#print("Performance correlations & RMSE")
condition <- c("MMM","MSM","SMS","SSS")
rmse <- rep(0,4)
for (cond in 1:4) {
    rmse[cond] <- RMSE(M1_Mat[,cond],H_Mat[,cond])
}
rmse_same <- rmse
rmse_same
rmse <- rep(0,4)
for (cond in 1:4) {
    rmse[cond] <- RMSE(M2_Mat[,cond],H_Mat[,cond])
}
rmse_diff <- rmse
rmse_diff

print("Overall correlation & RMSE")
M1_avg <- mean(rmse_same)
M2_avg <- mean(rmse_diff)
M1_avg
M2_avg

#Difference avg rmse delta - positive means keeping temperature is better
Mis_delta <- M2_avg - M1_avg
Mis_delta

[1] "Table for Misses"


[1] 3.980021 7.147168 7.879787 5.471934

[1] 3.894219 7.514872 8.005687 5.797483

[1] "Overall correlation & RMSE"


[1] 6.119728

[1] 6.303065

[1] 0.1833378